In [1]:
import os

os.environ['OPENAI_API_KEY'] = '****'
os.environ['LANGCHAIN_API_KEY'] = '****'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = '08-02'

### 1. 비휘발성 데이터 생성

In [2]:
!pip install -qU langchain_openai langchain_community faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.9/411.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.3 MB/s eta 0:00:00


In [12]:
from langchain.storage import LocalFileStore
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS

#OpenAI 임베딩을 사용하여 기본 임베딩 설정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

#로컬 파일 저장소 설정
store = LocalFileStore("./cache/")

#캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
  underlying_embeddings=embedding,
  document_embedding_cache=store,
  namespace=embedding.model, #기본 임베딩과 저장소를 사용하여 캐시 지원 임베딩을 생성
)

In [13]:
OpenAIEmbeddings().model

'text-embedding-ada-002'

In [14]:
# store에서 키들을 순차적으로 가져옴
list(store.yield_keys())

[]

In [15]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

#문서 로드
raw_documents = TextLoader("/content/data/appendix-keywords.txt").load()

#문자 단위로 텍스트 분할 설정
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

#문서 분할
documents = text_splitter.split_documents(raw_documents)

In [27]:
%time db = FAISS.from_documents(documents, cached_embedder)

CPU times: user 353 ms, sys: 49.5 ms, total: 402 ms
Wall time: 1.96 s


In [28]:
%time db2 = FAISS.from_documents(documents, cached_embedder)

CPU times: user 15.1 ms, sys: 788 µs, total: 15.9 ms
Wall time: 17.1 ms


### 2. 휘발성 DB 생성

In [36]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import InMemoryByteStore

store = InMemoryByteStore()  # 메모리 내 바이트 저장소 생성

# 캐시 지원 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    embedding, store, namespace=embedding.model
)



In [37]:
cached_embedder